<a href="https://colab.research.google.com/github/ShuriKimura0402/project-examples/blob/main/Flamingo_NVIDIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip

In [2]:
!pip install --upgrade git+https://github.com/huggingface/transformers accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ccglbtxy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ccglbtxy
  Resolved https://github.com/huggingface/transformers to commit cac0a28c83cf87b7a05495de3177099c635ba852
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
#Hugging Faceのtransformersライブラリから2つのクラスをインポートしています。AudioFlamingo3ForConditionalGenerationは音声言語モデル本体で、AutoProcessorは音声とテキストの前処理を自動で行うクラス
from transformers import AudioFlamingo3ForConditionalGeneration, AutoProcessor

In [4]:
#Hugging Face上のモデルIDを文字列として変数に格納しています。このIDを使ってモデルとプロセッサーをダウンロードします。
model_id = "nvidia/audio-flamingo-3-hf"

In [5]:
#指定したモデルID用のプロセッサーを自動的にダウンロードして初期化しています。このプロセッサーは音声ファイルやテキストを適切な形式に変換します。
processor = AutoProcessor.from_pretrained(model_id)

In [6]:
#Audio Flamingo 3モデル本体をダウンロードして初期化しています。device_map="auto"は利用可能なGPUに自動的にモデルを配置する設定です。
model = AudioFlamingo3ForConditionalGeneration.from_pretrained(model_id, device_map="auto")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/830 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
audio_path = "/content/Manaka_Elevenlabs.mp3"

In [8]:
#会話履歴を格納するリストを開始しています。このリストには、ユーザーとモデルのやり取りが含まれます。
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Transcribe the input speech."},
            {"type": "audio", "path": audio_path},
        ],
    }
]


In [9]:

inputs = processor.apply_chat_template( #プロセッサーのapply_chat_templateメソッドを呼び出して、会話をモデルが理解できる形式に変換し始めます。
    conversation, #先ほど定義した会話データを引数として渡しています。
    tokenize=True, #テキストをトークン(数値ID)に変換することを指定しています。
    add_generation_prompt=True, #モデルが応答を生成するためのプロンプト(合図)を自動的に追加します。
    return_dict=True,#結果を辞書形式で返すように指定しています(テンソルだけでなく、複数の情報を含む辞書が返されます)。
).to(model.device) #処理された入力データをモデルと同じデバイス(GPUまたはCPU)に移動させています。

In [10]:
#モデルに入力を渡して応答を生成しています。**inputsで辞書の内容を展開し、max_new_tokens=100で最大100トークンまで生成することを指定しています。
outputs = model.generate(**inputs, max_new_tokens=100)

In [11]:
#生成されたトークンIDを人間が読める文字列に変換しています。outputs[:, inputs.input_ids.shape[1]:]で入力部分を除いた生成部分だけを取り出し、skip_special_tokens=Trueで特殊トークン(例: <pad>, <eos>)を除去しています。
decoded_outputs = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

In [12]:
print(decoded_outputs)

["The spoken content of the audio is 'hey shirii, you know apple's char voice assistant, senpai is eliza, and the rival is amazon's alexa, so alexa, sometimes sensitive topics, inappropriate responses, personally, i don't think that's very good, there are many overseas customers, other language virtual assistants, income is the same as positive feedback, buzzed with x'."]
